In [ ]:
## Step 1: Install Required Python Packages

import os

# Setting environment variables for temporary directories
os.environ['TEMP'] = '/data/azee_env/sgnify_env/tmp'
os.environ['TEMPDIR'] = '/data/azee_env/sgnify_env/tmp'
os.environ['TMPDIR'] = '/data/azee_env/sgnify_env/tmp'

# Install the necessary Python packages.
!pip install bpy numpy tqdm
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
## Step 3: Extract Skeleton Data from a BVH File

# Here, we'll run the `extract_skeleton.py` script on one of the BVH files to generate the `skeleton.pt`.
import bpy
import numpy as np
import pickle
import mathutils
import math
import os

# Load one of the BVH files to extract the skeleton.
bvh_file = os.path.join(mocap_path, os.listdir(mocap_path)[0])
print(bvh_file)
bpy.ops.import_anim.bvh(filepath=bvh_file)

ob = bpy.context.object

if ob.type == 'ARMATURE':
    arm = ob.data

# Print the armature to verify.
print(arm)
        
bone_mapping = {}

num_bones = len(arm.bones)

# Bone's name to ID mapping.
for id, bone in enumerate(arm.bones):
    bone_mapping[bone.name] = id

# Bone's child-to-parent ID mapping.
kintree_table = []
for bone in arm.bones:
    this_bone_id = bone_mapping[bone.name]
    
    parent = bone.parent
    parent_id = bone_mapping[parent.name] if parent else -1
    kintree_table.append([parent_id, this_bone_id])

# Bone's LocalRestTransform.
J = []
for bone in arm.bones:
    if bone.parent:
        bonetrans = bone.parent.matrix_local.inverted() @ bone.matrix_local
    else:
        bonetrans = bone.matrix_local
    J.append([bonetrans.row[0].xyzw, bonetrans.row[1].xyzw, bonetrans.row[2].xyzw, bonetrans.row[3].xyzw])

body_data = {
    "J": np.array(J),
    "kintree_table": np.array(kintree_table).T,
    "name_to_id": bone_mapping
}

# Save the skeleton data.
os.makedirs("./data", exist_ok=True)
with open("./data/skeleton.pt", "wb") as f:
    pickle.dump(body_data, f)

print("Skeleton data has been extracted and saved to ./data/skeleton.pt")

In [ ]:
## Step 3: OR Download Skeleton Data
import os

os.makedirs("./data", exist_ok=True)
!wget -O skeleton.pt https://nextcloud.lisn.upsaclay.fr/index.php/s/H27FBnZFCcAmNzC/download/skeleton.pt
!mv skeleton.pt ./data/skeleton.pt

In [ ]:
## Step 4: Extract Animation Data from BVH Files

# This step processes each BVH file and extracts the animation data using the `bpy_import_bvh_and_convert.py` script.

import bpy
import numpy as np
import pickle
import sys
import os
import tqdm

# Function to extract the frame count from the BVH file
def extract_frame_count(bvh_file):
    with open(bvh_file, 'r') as file:
        for line in file:
            if "Frames:" in line:
                return int(line.split()[1])
    return 0

# Function to process a single BVH file
def process_bvh_file(bvh_file, mapping, output_dir):
    print("Processing:", bvh_file)
    
    # Extract the frame count from the BVH file
    frame_count = extract_frame_count(bvh_file)
    print(f"Extracted frame count from BVH file: {frame_count}")

    # Import the BVH file
    bpy.ops.import_anim.bvh(filepath=bvh_file)

    # Access the imported armature object and scene
    ob = bpy.context.object
    sce = bpy.context.scene

    # Ensure the object is an armature
    if ob and ob.type == 'ARMATURE':
        armature = ob

        # Set the scene's end frame to the frame count extracted from the BVH file
        sce.frame_end = frame_count
        print(f"Setting scene frame_end to: {frame_count}")

        anim_data = np.zeros((frame_count, len(mapping.keys()), 3, 3))

        # Iterate over each frame and extract pose data
        for f in range(1, frame_count + 1):
            sce.frame_set(f)
            
            for pbone in armature.pose.bones:
                mat_local = np.array(pbone.matrix_basis)[:3, :3]
                pbone_id = mapping.get(pbone.name, None)
                if pbone_id is not None:
                    anim_data[f-1, pbone_id, :, :] = mat_local

        print("Animation Data Shape:", anim_data.shape)

        # Ensure the 'extracted' directory exists, if not, create it
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save the extracted animation data
        output_file = os.path.join(output_dir, "{}.pkl".format(armature.name))
        with open(output_file, "wb") as f:
            pickle.dump(anim_data, f)
    else:
        print("Cannot find armature")

output_dir = "./dataset/extracted"

# Load the skeleton mapping
with open("./data/skeleton.pt", "rb") as f:
    mapping = pickle.load(f)["name_to_id"]

# Collect all BVH files
mocap_files = [os.path.join(mocap_path, file) for file in os.listdir(mocap_path) if file.endswith(".bvh")]

# Process each BVH file
for mocap_file in tqdm.tqdm(mocap_files):
    process_bvh_file(mocap_file, mapping, output_dir)

print("All BVH files have been processed.")

In [ ]:
# Download the BVH files from the provided link.
#!wget -O extracted.zip https://nextcloud.lisn.upsaclay.fr/index.php/s/7cZsL6oXJ6NekDM/download
!wget -O extracted.zip https://nextcloud.lisn.upsaclay.fr/index.php/s/qGXAEcjCWYS57rG/download
!unzip extracted.zip -d ./dataset

In [ ]:
## Step 5: Prepare Data for VPoser Training

# Clone the repository if not already cloned
repo_url = "https://github.com/Paritosh97/latent-pose-sl.git"
repo_dir = "./latent_pose_sl"
if not os.path.exists(repo_dir):
    !git clone {repo_url} {repo_dir}

module_path = os.path.abspath(os.path.join('nnutils'))

import os
import pickle
import numpy as np
import torch
from tqdm import tqdm
from latent_pose_sl.nnutils.vposer import VPoser  # Adjust the import if the path changes

# Path to the extracted animation data
extracted_anim_path = "./dataset/extracted/"
if not os.path.exists(extracted_anim_path):
    raise FileNotFoundError(f"The directory {extracted_anim_path} does not exist.")

# List all the .pkl files in the directory
extracted_anim_files = [
    os.path.join(extracted_anim_path, file) 
    for file in os.listdir(extracted_anim_path) 
    if file.endswith(".pkl")
]

# Check if there are any files to process
if not extracted_anim_files:
    raise FileNotFoundError("No .pkl files found in the extracted animation directory.")

# Process the animation data
poses = []
for anim_file in tqdm(extracted_anim_files, desc="Processing animation files"):
    with open(anim_file, "rb") as f:
        anim_data = pickle.load(f)
        anim_data = torch.tensor(anim_data, dtype=torch.float32)
        anim_data = anim_data.reshape((anim_data.shape[0], 1, anim_data.shape[1], 9))
        aa = VPoser.matrot2aa(anim_data)  # Ensure this is the correct method and usage
        poses.append(aa)
poses = torch.cat(poses, dim=0).squeeze(dim=1).numpy()

# Shuffle and split the data
np.random.shuffle(poses)
lens = len(poses)
pose_train, pose_val, pose_test = (
    poses[:int(0.7 * lens)],
    poses[int(0.7 * lens):int(0.9 * lens)],
    poses[int(0.9 * lens):],
)

# Save the data
save_dir = "./data/train"
os.makedirs(save_dir, exist_ok=True)
torch.save(torch.from_numpy(pose_train), os.path.join(save_dir, "pose_train.pt"))
torch.save(torch.from_numpy(pose_val), os.path.join(save_dir, "pose_val.pt"))
torch.save(torch.from_numpy(pose_test), os.path.join(save_dir, "pose_test.pt"))

print("Training data has been prepared and saved.")

%cd latent_pose_sl

In [ ]:
import sys

# Define the custom paths for the dataset and skeleton file
custom_datapath = "../data/train/pose"  # Replace with your actual dataset path
custom_skeletonpath = "../data/skeleton.pt"  # Replace with your actual skeleton file path
num_epochs = 300  # Specify the number of epochs if you want to change the default


module_path = os.path.abspath(os.path.join('nnutils'))

sys.path.insert(0, module_path)

import torch

# Check if a GPU is available
if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    # Print the name of the GPU
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Using CPU.")
    
!mkdir snapshots

# Run the training script with custom paths
!python nnutils/train_vposer.py --datapath $custom_datapath --skeletonpath $custom_skeletonpath --epochs $num_epochs

In [ ]:
from IPython.display import FileLink

# Path to the best model file
model_path = "./snapshots/E250.pt"

# Create a link to download the file
FileLink(model_path)